In [2]:
import pandas as pd
import numpy as np
import random

# Parameters
num_rows = 1_000_000  # Number of rows
start_date = "2021-01-01"
end_date = "2023-12-31"


## Kategorien und Preisspannen

In [4]:
# Expanded categories and items
categories = {
    "Soft Drinks": ["Coca-Cola", "Pepsi", "Fritz-Kola", "Sprite", "Fanta", "Mezzo Mix", "Schwip Schwap", 
                    "Mountain Dew", "Club Mate", "Red Bull", "Monster", "Rockstar"],
    "Alcoholic Beverages": ["Beck's", "Warsteiner", "Erdinger Weißbier", "Jever", "Krombacher", "Augustiner",
                            "Kölsch", "Riesling", "Merlot", "Chardonnay", "Sauvignon Blanc", 
                            "Johnnie Walker", "Jack Daniels", "Bacardi", "Havana Club", "Tanqueray",
                            "Moët & Chandon", "Veuve Clicquot", "Rotkäppchen Sekt"],
    "Water": ["Vittel", "Volvic", "Evian", "Gerolsteiner", "San Pellegrino", "Apollinaris", "Selters", "Volvic Touch", "Vio Wasser"],
    "Juices": ["Hohes C Orange", "Granini Apple", "Tomato Juice", "Cranberry Juice", "Rauch Multivitamin", 
               "Mango Juice", "Passion Fruit Juice"]
}

# Market price ranges
price_ranges = {
    # Soft Drinks
    "Coca-Cola": (0.5, 1.5), "Pepsi": (0.5, 1.5), "Fritz-Kola": (1.5, 2.5), "Sprite": (0.5, 1.5),
    "Fanta": (0.5, 1.5), "Mezzo Mix": (0.5, 1.5), "Schwip Schwap": (0.5, 1.5), 
    "Mountain Dew": (0.5, 1.5), "Club Mate": (1.5, 2.5), "Red Bull": (1.5, 3.5),
    "Monster": (1.5, 3.5), "Rockstar": (1.5, 3.5),
    
    # Alcoholic Beverages
    "Beck's": (0.8, 2.0), "Warsteiner": (0.8, 2.0), "Erdinger Weißbier": (1.0, 3.0),
    "Jever": (0.8, 2.0), "Krombacher": (0.8, 2.0), "Augustiner": (1.0, 3.0),
    "Kölsch": (0.8, 2.5), "Riesling": (3.0, 15.0), "Merlot": (3.0, 15.0), 
    "Chardonnay": (3.0, 15.0), "Sauvignon Blanc": (3.0, 15.0), 
    "Johnnie Walker": (15.0, 50.0), "Jack Daniels": (15.0, 50.0), 
    "Bacardi": (10.0, 40.0), "Havana Club": (10.0, 40.0), "Tanqueray": (15.0, 50.0), 
    "Moët & Chandon": (20.0, 100.0), "Veuve Clicquot": (20.0, 100.0), "Rotkäppchen Sekt": (3.0, 10.0),
    
    # Water
    "Vittel": (0.3, 1.0), "Volvic": (0.3, 1.0), "Evian": (0.5, 1.5),
    "Gerolsteiner": (0.3, 1.5), "San Pellegrino": (0.3, 1.5), "Apollinaris": (0.3, 1.5),
    "Selters": (0.3, 1.5), "Volvic Touch": (0.5, 2.0), "Vio Wasser": (0.5, 2.0),
    
    # Juices
    "Hohes C Orange": (1.0, 3.0), "Granini Apple": (1.0, 2.5), "Tomato Juice": (1.5, 3.5),
    "Cranberry Juice": (2.0, 4.0), "Rauch Multivitamin": (1.5, 3.5),
    "Mango Juice": (2.0, 4.0), "Passion Fruit Juice": (2.0, 4.0)
}

# Validation: Ensure all products in categories are in price_ranges
missing_products = [
    product for category_products in categories.values() for product in category_products if product not in price_ranges
]

if missing_products:
    print(f"Fehlende Preisspannen für folgende Produkte: {missing_products}")
else:
    print("Alle Produkte haben gültige Preisspannen!")

# Generate B2C prices for all products
b2c_prices = {product: round(np.random.uniform(*price_ranges[product]), 2) for product in price_ranges.keys()}


Alle Produkte haben gültige Preisspannen!


## Weitere Konfigurationen

In [6]:
# Initial fixed prices for B2C customers
b2c_prices = {product: round(np.random.uniform(*price_ranges[product]), 2) for product in price_ranges.keys()}

# Premium brands
premium_brands = {"Moët & Chandon", "Veuve Clicquot", "Johnnie Walker", "Fritz-Kola", "Evian"}

# Regional preferences
regional_preferences = {
    "Bayern": {"Erdinger Weißbier": 0.4, "Hohes C Orange": 0.3},
    "Schleswig-Holstein": {"Jever": 0.4, "Gerolsteiner": 0.3},
    "Berlin": {"Fritz-Kola": 0.5, "Sprite": 0.4},
    "Nordrhein-Westfalen": {"Kölsch": 0.4, "Fanta": 0.4},
    "Hessen": {"Riesling": 0.3, "Hohes C Orange": 0.4},
    "Baden-Würtemberg":{}
}


## Kunden

In [11]:
# Anzahl der Kunden
num_customers = 10_000

type = ["B2B", "B2C"]
weights = [0.3, 0.7]

regions = list(regional_preferences.keys())

# Zufällige Verteilung der Kundentypen
customer_types = {f"CUS{customer_id}": random.choices(type, weights=weights, k=1)[0] for customer_id in range(1, num_customers + 1)}
customer_regions = {f"CUS{customer_id}": random.choice(regions) for customer_id in range(1, num_customers + 1)}

## Funktionen

In [24]:
# Define order date
def calculate_date(start_date, end_date):
    days = np.random.randint(0, (end_date - start_date).days)
    return start + pd.to_timedelta(days, unit="D")

# Split Price ranges in low, middle, high ranges
def split_price_ranges(product):
    split_range = {}
    low, high, =  price_ranges[product]
    diff = high - low
    split_range = {
            "low": (low, low + diff / 3),
            "middle": (low + diff / 3, low + 2 * diff / 3),
            "high": (low + 2 * diff / 3, high)
        }
    return split_range

def calculate_price_elements(customer_type, product, year):

    # Inflation of 3% per year
    years = (year - 2021) 
    inflation = 1.03 ** years 

    # B2B customer
    if customer_type == "B2B":
        # Quantity
        quantity = random.randint(1, 100) # Quantity 1 - 100
        weights = [max(100 - quantity, 10), max(quantity, 20), max(quantity - 50, 5)] # adjusting weighting with higher quantity
        weights = np.array(weights) / sum(weights) # normalizing for 100  
        
        # Discount      
        discounts = [5, 10, 15] # List of possible discounts
        discount = np.random.choice(discounts, p=weights) / 100 # choosing discount
        
        # Price
        ranges = split_price_ranges(product) # Splitting price range in low, middle, high
        price_range = np.random.choice(["low", "middle", "high"], p=weights) # selecting range dependant from quantity
        selected_price = round(np.random.uniform(*ranges[price_range]), 2) # Random price from range
        price = selected_price / (1 - discount) # adding discount 

    # B2C customer    
    else:
        quantity = random.randint(1, 15) # Quantity 1 - 15
        price = b2c_prices[product] # Choosing fixed price without range
        discount = 0 # no discount in B2C

    price = round(price*inflation, 2) # Rounding 
    
    return quantity, price, discount 

def adjust_price(base_price, product, month):
    adjusted = False  # Trac changes
    if month in [6, 7, 8] and product in ["Coca-Cola", "Pepsi", "Beck's", "Erdinger Weißbier", "Vittel"]:
        base_price *= 1.1  # 10% increase in summer
        adjusted = True
    if month == 12 and product in ["Moët & Chandon", "Riesling"]:
        base_price *= 1.2  # 20% increase for Christmas
        adjusted = True
    if product in premium_brands:
        base_price *= 1.15  # 15% increase for premium brands
        adjusted = True
    if product in ["Red Bull", "Monster", "Rockstar"]:
        base_price *= 1.05  # 5% increase for high-demand products
        adjusted = True

    return round(base_price, 2) if adjusted else base_price
    

# Function to calculate total price
def calculate_total_price(unit_price, quantity, discount):
    return round(unit_price * quantity * (1 - discount), 2)

## Datengenerierung

In [26]:
data = []
max_items_per_order = 5 
start = pd.to_datetime(start_date)
end = pd.to_datetime(end_date)

for i in range(num_rows):
    order_id = f"ORD{i + 1}"  # Unique Order_ID
    customer_id = random.choice(list(customer_types.keys())) # Random Customer
    customer_type = customer_types[customer_id]   # Customer type
    region = customer_regions[customer_id] # Customer region
    order_date = calculate_date(start, end) # Calculate random order date in Daterange
    year = order_date.year 
    month = order_date.month

    num_items = random.randint(1, max_items_per_order)
    selected_products = set()
    
    for _ in range(num_items):
        category = random.choice(list(categories.keys())) # Random choice of a product category

        # Choosing product
        available_products = list(set(categories[category]) - selected_products)
        if not available_products:
            break  # No more available products left

        
        preferences = regional_preferences.get(region, {}) # Getting regional preferences
        product_weights = [preferences.get(product, 1 - sum(preferences.values())) for product in available_products] # Calculation preferences for each product
        total_weight = sum(product_weights) # Normalizing the weighting
        product_weights = [w / total_weight for w in product_weights]       
        product = random.choices(available_products, weights=product_weights, k=1)[0] # Choosing product 
        selected_products.add(product) # Adding product to the list
        
        quantity, unit_price, discount = calculate_price_elements(customer_type, product, year) # Calculating quantity, price and discount 

        # Apply changes to the unit price dependant on rules
        unit_price = adjust_price(unit_price, product,month)

        # Calculate final price 
        total_price = round(unit_price * quantity * (1-discount), 2)

        data.append([order_id, customer_id, customer_type, product, category, unit_price, quantity, discount, total_price, region, order_date])


In [30]:
columns = ["Order_ID", "Customer_ID", "Customer_Type", "Product", "Category", "Unit_Price", 
           "Quantity", "Discount", "Total_Price", "Region", "Order_Date"]

df = pd.DataFrame(data, columns=columns)
df.head()

,Order_ID,Customer_ID,Customer_Type,Product,Category,Unit_Price,Quantity,Discount,Total_Price,Region,Order_Date
0,ORD1,CUS506,B2B,Veuve Clicquot,Alcoholic Beverages,105.31,61,0.10,5781.52,Berlin,2022-08-08
1,ORD1,CUS506,B2B,Volvic Touch,Water,1.35,80,0.15,91.80,Berlin,2022-08-08
2,ORD1,CUS506,B2B,Beck's,Alcoholic Beverages,1.45,37,0.05,50.97,Berlin,2022-08-08
3,ORD1,CUS506,B2B,Hohes C Orange,Juices,2.15,17,0.05,34.72,Berlin,2022-08-08
4,ORD1,CUS506,B2B,Volvic,Water,0.84,92,0.10,69.55,Berlin,2022-08-08


## DataFrame und Speicherung

In [31]:
# Save to CSV
df.to_csv("synthetic_beverage_sales_data.csv", index=False)
print("Dataset with unique Order_IDs created!")


Dataset with unique Order_IDs created!
